In [1]:
import os

In [2]:
%pwd

'c:\\Users\\User\\Desktop\\Project Anchor\\End-to-End-ML-with-MLflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\User\\Desktop\\Project Anchor\\End-to-End-ML-with-MLflow'

In [20]:
from dataclasses import dataclass
from pathlib import Path

# Data Transformation configuration entity
@dataclass(frozen=True)
class DataTransformationConfig:
    """
    Configuration class for Data Transformation.

    Attributes:
    - root_dir (Path): The root directory for data transformation.
    - data_path (Path): The path to the data.
    """
    root_dir: Path
    data_path: Path


In [21]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories

In [22]:
# configuration manager for src
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [23]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [24]:
#components
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        """
        Constructor for DataTransformation class.

        Args:
            config (DataTransformationConfig): Configuration object for data transformation.
        """
        self.config = config

        # NB: We can add several data transformation techniques such as scalar, PCA, and all.
        # We can perform all kinds of EDA in the ML cycle here before passing the data to the model,
        # but since I'm using clean data, I'll proceed.

    def train_test_split(self):
        """
        Splits the data into training and test sets.
        """
        # Read data from CSV file
        data = pd.read_csv(self.config.data_path)

        # Split the data into training and test sets (0.75, 0.25).
        train, test = train_test_split(data, test_size=0.25, random_state=42)

        # Save the split data to CSV files.
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        # Log information about the split data.
        logger = logging.getLogger(__name__)
        logger.info("Split data into training and test sets")
        logger.info(f"Training data shape: {train.shape}")
        logger.info(f"Test data shape: {test.shape}")

        # Print information about the split data.
        print(f"Training data shape: {train.shape}")
        print(f"Test data shape: {test.shape}")


In [26]:
# Update Pipeline
try:
    # Load configuration manager to retrieve data transformation configuration
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()

    # Instantiate DataTransformation object with the retrieved configuration
    data_transformation = DataTransformation(config=data_transformation_config)
    
    # Perform train-test splitting as part of the data transformation process
    data_transformation.train_test_split()

except Exception as e:
    # Raise any exceptions that occur during the pipeline execution for further handling
    raise e


[2024-03-08 17:25:08,295: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-08 17:25:08,296: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-08 17:25:08,310: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-08 17:25:08,313: INFO: common: created directory at: artifacts]
[2024-03-08 17:25:08,315: INFO: common: created directory at: artifacts/data_transformation]
[2024-03-08 17:25:08,388: INFO: 1621353329: Split data into training and test sets]
[2024-03-08 17:25:08,388: INFO: 1621353329: Training data shape: (1199, 12)]
[2024-03-08 17:25:08,388: INFO: 1621353329: Test data shape: (400, 12)]
Training data shape: (1199, 12)
Test data shape: (400, 12)
